In [1]:
from shapely.ops import nearest_points
from shapely.geometry import Point
import geopandas as gpd

print(nearest_points(Point(0,2), Point(1,4))[1])  #Crea tupla in cui butta primo e secondo numero

A=gpd.GeoDataFrame()
A["geometry"]=[Point(1,4), Point(0,3)]

##Serve passare per dataframe per fare unary union (non so how altrimenti, sicuro c'è modo)
u=nearest_points(Point(0,2), A.geometry.unary_union)[1]
print(u)

nearest=A["geometry"]==u
print(nearest)

print(A[nearest])  
#print(A[u])

#Metti tutto insieme, e trovi la stazione giusta

#Giusto come nota:  stations[[True]*20+[False]*16]["station"].values[0]

POINT (1 4)
POINT (0 3)
0    False
1     True
Name: geometry, dtype: bool
                  geometry
1  POINT (0.00000 3.00000)


In [35]:
A.geometry

0      POINT (1.00000 4.00000)
1      POINT (0.00000 3.00000)
2      POINT (1.00000 4.00000)
3      POINT (0.00000 3.00000)
4      POINT (1.00000 4.00000)
                ...           
795    POINT (0.00000 3.00000)
796    POINT (1.00000 4.00000)
797    POINT (0.00000 3.00000)
798    POINT (1.00000 4.00000)
799    POINT (0.00000 3.00000)
Name: geometry, Length: 800, dtype: geometry